## Continued Fractions Algorithm

The Continued Fractions Algorithm finds the continued fraction expansion for any irrational number to as many partial denominators as desired

The following system of equations is a way of describing the continued fractions algorithm

$$
\begin{align*}
   x &= a_0 + b_0 \\
   \frac{1}{b_0} &= a_1' = a_1 + b_1 \\
   &\vdots\\
   \frac{1}{b_k} &= a_{k+1}' = a_{k+1} + b_{k+1}
\end{align*}
$$

Source: [Cornell](https://pi.math.cornell.edu/~gautam/ContinuedFractions.pdf)

---

**Algorithm ContinuedFraction(x)**
 
**Input:** A number $x \in \mathbb{R}$

**Output:** Continued fraction representation of $x$: $\left[ a_0, a_1, a_2, ... \right]$ such that


**begin**

1. $a_m := \lfloor$ (i.e., let $a_m$ be the integer part of $x_m$)
2. $b_m := x_m - a_m$
3. $\if{b_m \neq 0}$
4. &emsp; Set $x_{m+1} = 1/b_m$ and go back to step 1 to compute $a_{m+1}$
5. $\else$
6. &emsp; return

**end.**

Note: as some sequences can grow very large, we define a positive $\epsilon \ll 1$ so that instead of directly testing $b_m = 0$, we instead test $|b_m| < \epsilon$. This will allow us to check if $b_m$ is approximately zero.

---

In [1]:
import numpy as np

def ContinuedFraction(x, eps=1e-8, retlist=False):
    """
    Return approximate fraction representation of a value.

    Parameters
    ----------
        eps : int, optional
            The threshold for approximating bm to 0. Default: 1e-8.
        retlist : bool, optional
            if True, also returns the list of coefficients for the representation.

    Returns
    -------
        numr: int
            The numerator of the approximated fraction.
        denr: int
            Only returned is `retlist` is False
            
            The denominator of the approximated fraction.
        coeffs: ndarray
            Only returned if `retlist` is True

            Coefficient representation of the approximated fraction.
    """
    
    def BuildContinuedFractionRepresentation(xm, a: np.ndarray):
        # Compute the integer part of xm
        am = int(xm)                      
        
        # Add integer part of xm to the results list (denominator of continued fractions)                                
        a = np.append(a, am)
        
        # Compute denominator of x_{m+1}                                  
        bm = xm - am                                                      
        
        # Terminate if bm ~ 0 
        if np.abs(bm) < eps:
            return a
        else:
            # Compute a_{m+1}
            return BuildContinuedFractionRepresentation(1/bm, a)

    # Get coefficients of approximated fraction 
    coeffs = BuildContinuedFractionRepresentation(x, np.array([], dtype=int))

    # https://math.stackexchange.com/questions/3084970/how-to-convert-continued-fractions-into-normal-fractions
    H = np.empty(len(coeffs) + 2)
    K = np.empty_like(H)
    H[0], H[1] = 0, 1
    K[0], K[1] = 1, 0
    for n in range(2, len(H)):
        H[n] = coeffs[n-2] * H[n-1] + H[n-2]
        K[n] = coeffs[n-2] * K[n-1] + K[n-2]

    numr = H[-1]
    denr = K[-1]

    if retlist:
        return numr, denr, coeffs
    
    return numr, denr

    
# =================================
# tests
# =================================

# 23/8 = 2.875
assert (ContinuedFraction(2.875) == (23, 8))
assert (ContinuedFraction(2.875, retlist=True)[2] == [2,1,7]).all()

# 27/4 = 6.75
assert (ContinuedFraction(6.75) == (27,4))
assert (ContinuedFraction(6.75, retlist=True)[2] == [6,1,3]).all()

print('All tests passed')

All tests passed


$\newcommand{\if}[1]{{\textbf{if }{#1}\textbf{ then:}}}$
$\newcommand{\else}{{\textbf{else:}}}$

# Shor's Algorithm

Shor's algorithm accepts a (large) integer $N$ and finds $p$, $q$ such that $N=p\cdot q$ and both $p$, $q$ are prime

---

**Algorithm Shor**

**Input:** The integer to factor, $N$

**Output:** $p,q$ such that $N = p \cdot q$

**begin**

1. Choose a random integer $1 < a < N$
2. $b := \gcd(a,N)$
3. $\if{b \neq 1}$
4. &emsp;$b$ factors $N$; return $p=b$ and $q=N/b$
5. $\else$
6. &emsp;Use quantum subroutine to find the order $r$ of $a$
7. &emsp;$\if{r\text{ is odd}}$
8. &emsp;&emsp;go to line 1
9. &emsp;$g := \gcd(a^{r/2}\pm 1, N)$
10. &emsp;$\if{g>1}$
11. &emsp;&emsp;return $p=g$, $q=N/g$
12. &emsp;$\else$
13. &emsp;&emsp;go to line 1

**end.**

---


# Modular Exponentiation

This will describe how the unitary operator $U$ (to be used in the quantum period-finding subroutine) will be constructed.

---

**Algorithm QModularExponentiation**

**Input:** 
- Integers $a,N$: same values from **Algorithm Shor**
- Integer $x$
- Integers $n,m$: the number of qubits being used in each register

**Output:** Operator $U$ such that 

$$
U\left( \frac{1}{\sqrt{2^{n}}} \sum_{x \in \{ 0,1 \}^{n}} \ket{x} \ket{0}^{\otimes m} \right) = \frac{1}{\sqrt{2^{n}}} \sum_{x \in \{ 0,1 \}^{n}} \ket{x} \ket{a^x \bmod{N}}^{\otimes m}
$$

**begin**

1. Write $x$ in its binary form $x = x_{n-1} x_{n-2} \dots x_{0} = x_{n-1}2^{n-1} + x_{n-2}2^{n-2} + \dots + x_{0}2^{0} = \sum_{k=0}^{n-1} x_k 2^k$

&emsp;&emsp;(Note that doing this allows us to use $a^x \bmod{N} = a^{x_{n-1}2^{n-1} + x_{n-2}2^{n-2} + \dots + x_{0}2^{0}} \bmod{N} = \left( (a^{x_{n-1}2^{n-1}}\bmod{N})(a^{x_{n-2}2^{n-2}}\bmod{N}) \dots (a^{x_0 2^0} \bmod{N})\right) \bmod{N}$)

2. $G := G_0, G_1, \dots, G_{n-1}; G_0 = a^{x_0}$
3. Populate $G$ such that $G_{j} = G_{j-1} \left( a^{x_j 2^j} \bmod{N} \right)$

**end.**

---

In [35]:
import math
from qiskit import QuantumCircuit

def IntToBytes(n: int, num_bits: int = None) -> list:
    nbits = int( math.log(n,2) + 1 ) if n > 0 else 0
    if num_bits != None: nbits = num_bits

    return [(n >> bit) & 1 for bit in range(nbits - 1, -1, -1)]

def c_amodN(a: int, N: int, x: int, nqb: int):
    U = QuantumCircuit(nqb)
    Xs = IntToBytes(x, nqb)[::-1] # reversed for convenience on indexing

    #print(f'x = {x} = {Xs[::-1]} = {np.sum([xi * 2**i for i,xi in enumerate(Xs)])}')
    
    G = np.empty(nqb)
    #   base case
    G[0] = a**Xs[0]        
    #   populate G
    for j in range(1, nqb):
        #print(f'x_{j} = {Xs[j]} so multiply by {a**(Xs[j] * 2**j)} mod {N} = {a**(Xs[j] * 2**j) % N}')
        G[j] = G[j-1] * (a**(Xs[j] * 2**j) % N)
    G %= N

    print(f'expecting {a}^{x} mod {N} = {(a**x) % N}')
    return G

for mpl in range(40):
    print(f'{c_amodN(7, 15, mpl, 7)}\n')

expecting 7^0 mod 15 = 1
[1. 1. 1. 1. 1. 1. 1.]

expecting 7^1 mod 15 = 7
[7. 7. 7. 7. 7. 7. 7.]

expecting 7^2 mod 15 = 4
[1. 4. 4. 4. 4. 4. 4.]

expecting 7^3 mod 15 = 13
[ 7. 13. 13. 13. 13. 13. 13.]

expecting 7^4 mod 15 = 1
[1. 1. 1. 1. 1. 1. 1.]

expecting 7^5 mod 15 = 7
[7. 7. 7. 7. 7. 7. 7.]

expecting 7^6 mod 15 = 4
[1. 4. 4. 4. 4. 4. 4.]

expecting 7^7 mod 15 = 13
[ 7. 13. 13. 13. 13. 13. 13.]

expecting 7^8 mod 15 = 1
[1. 1. 1. 1. 1. 1. 1.]

expecting 7^9 mod 15 = 7
[7. 7. 7. 7. 7. 7. 7.]

expecting 7^10 mod 15 = 4
[1. 4. 4. 4. 4. 4. 4.]

expecting 7^11 mod 15 = 13
[ 7. 13. 13. 13. 13. 13. 13.]

expecting 7^12 mod 15 = 1
[1. 1. 1. 1. 1. 1. 1.]

expecting 7^13 mod 15 = 7
[7. 7. 7. 7. 7. 7. 7.]

expecting 7^14 mod 15 = 4
[1. 4. 4. 4. 4. 4. 4.]

expecting 7^15 mod 15 = 13
[ 7. 13. 13. 13. 13. 13. 13.]

expecting 7^16 mod 15 = 1
[1. 1. 1. 1. 1. 1. 1.]

expecting 7^17 mod 15 = 7
[7. 7. 7. 7. 7. 7. 7.]

expecting 7^18 mod 15 = 4
[1. 4. 4. 4. 4. 4. 4.]

expecting 7^19 mod 15 = 13
[

## Quantum Period Finding Algorithm

---

**Algorithm QCalculatePeriod**

**Input:** Coprime integers $N$ and $1 \lt a \lt N$

**Output:** The order of $a$ (i.e., $\min(r>0)$ such that $a^r = 1\bmod{N}$)

**begin**

1. $m := \lceil \lg N \rceil$
2. $n := 2m$
3. Prepare two registers, one of size $n$ and the other of size $m$: $\ket{\Psi} = \ket{0}^{\otimes n}\ket{\psi}^{\otimes m}$
4. Apply a Hadamard transform to the first register in $\ket{\Psi}$
5. Apply a unitary operator $U$ to the second register, where $U\ket{\psi} = e^{i2 \pi \theta}\ket{\psi}$
6. Compute the inverse Quantum Fourier Transform on the first register
7. Measure the first register to find the most probable $k$ such that $k=qN/r$ ($q$ is a random integer $\in \left[ 0, r \right)$
8. Compute the continued fraction approximation of $N/k$. It is likely that this approximation will be an integer multiple of the period $r$.

**end.**

Note: $n$ is proportional to the accuracy of the algorithm; $n$ is arbitrary and does not necessarily have to be defined as $2m$ in Step 2.

---

In [4]:
from qiskit.circuit.library import QFT
from qiskit import ClassicalRegister, QuantumRegister

def funclog(msg: str, debug):
    if debug: print(msg)

def BuildPeriodFindingCircuit(a, N, eps=1e-1, debug=True):
    L = 1 + 2 * math.ceil( math.log(N, 2) )
    m = L + math.ceil( math.log(2 + 1/2/eps, 2) )
    m = 8
    n = 2*m
    
    funclog(f'n={n}, m={m}', debug)

    qr1 = QuantumRegister(n)
    qr2 = QuantumRegister(m)
    # Classical register for storing measurements
    crr = ClassicalRegister(n)

    # Create circuit
    circ = QuantumCircuit(qr1, qr2, crr)

    funclog('initialized circuit', debug)

    # apply Hadamard gate to first register
    funclog('applying Hadamard...', debug)
    circ.h(qr1)
    circ.x(-1)
    circ.barrier()

    
    # apply the Unitary operator to second register
    # --------------------------------------------------------
    funclog('applying U...', debug)
    
    def c_amodN(a, x, num_qubits):
        U = QuantumCircuit(num_qubits)
        Xs = IntToBytes(x, num_qubits)[::-1] # reversed for convenience on indexing
        G = np.empty(num_qubits + 1)
        #   base case
        G[0] = a**Xs[0]        
        #   populate G
        for i,xi in enumerate(Xs):
            G[i+1] = G[i] * (a**(xi * 2**i) % N)
        G %= N

        print(f'G = {G}')
        print(f'X = {Xs} (x={x})\n\n')
        return
        
        for i,xi in enumerate(Xs):
            G[i+1] = (G[i]**2 if xi != 0 else G[i]) 
            # T[i] = (T[i-1] * G[i]**xi) % N
            for q in range(num_qubits):
                U.x(q)
        U = U.to_gate()
        U.name = f'{a}^{x} mod N'
        return U.control()
        
    for x in range(2**n):
        cir = c_amodN(a, x, n)
        #circ.append(cir, [q] + [i+n for i in range(m)])
        
    # --------------------------------------------------------
    circ.barrier()

    #Apply inverse QFT to the first Register
    funclog('applying IQFT...', debug)
    circ.append(QFT(n, inverse=True), qr1)
    circ.barrier()
    
    #Measure the first register 
    funclog('measuring...', debug)
    circ.measure(qr1, crr)
    
    return circ, n, m

In [81]:
a, N = 7, 15

circ, n, m = BuildPeriodFindingCircuit(a, N)

#Execute the ciruit 
circ.draw(fold=-1)

n=16, m=8
initialized circuit
applying Hadamard...
applying U...
G = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=0)


G = [7. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
X = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=1)


G = [1. 1. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
X = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=2)


G = [7. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=3)


G = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=4)


G = [7. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
X = [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=5)


G = [1. 1. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
X = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] (x=6)


G = [7. 4. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
X = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

KeyboardInterrupt: 

In [4]:
from qiskit import transpile
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import Aer, AerSimulator
from qiskit_aer.primitives import SamplerV2

simulator = AerSimulator()
circuit = transpile(circ, simulator)

result = simulator.run(circuit, shots=10, memory=True).result()
memory = result.get_memory()
phase = int(memory[0],2)/(2**(n-1))
memory, phase, ContinuedFraction(phase)

(['1000000000',
  '1000000000',
  '1000000000',
  '1000000000',
  '0000000000',
  '1000000000',
  '0000000000',
  '1000000000',
  '1000000000',
  '0000000000'],
 1.0,
 (np.float64(1.0), np.float64(1.0)))